# **Homework**
1. 使用其他的线上文档或离线文件，重新构建向量数据库，尝试提出3个相关问题，测试 LCEL 构建的 RAG Chain 是否能成功召回。
2. 重新设计或在 LangChain Hub 上找一个可用的 RAG 提示词模板，测试对比两者的召回率和生成质量。

In [1]:
# 导入必要的库
import bs4,pdfplumber
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

### **Step1 加载文档**

In [2]:
# 使用pdfplumber加载本地 PDF 文档并提取文本内容
with pdfplumber.open("./testdocs/drawingtest.pdf") as pdf:
    content = ""
    for page in pdf.pages:
        content += page.extract_text() + "\n"

### **Step2 分割文档**

In [3]:
# 自定义分割器类，按 '---' 分割内容
class QATextSplitter:
    def split_text(self, text):
        # 按 '---' 分割文本
        return text.split('---')

In [4]:
# 去除多余的空行和分页符，合并换行符
content = content.replace("\n\n", "") # 去除多余的换行
content = content.replace("\x0c", "")  # 去除分页符

# 使用自定义文本分割器
text_splitter = QATextSplitter()

# 使用自定义分割器进行分割
chunks = text_splitter.split_text(content)

# 过滤掉空白块或仅包含空白字符的块
qa_chunks = [chunk.strip() for chunk in chunks if chunk.strip()]

In [5]:
# 打印分割后的块总数和部分内容，以便检查
print(f"总块数: {len(qa_chunks)}")

总块数: 250


In [6]:
# 输出分割后的块信息（打印前10个块）
for i, chunk in enumerate(qa_chunks[:10]):
    print(f"块 {i+1} 内容:\n{chunk}\n")


块 1 内容:
001
问题：你们的美术课程适合多大的孩子？
回答：我们的美术课程专为4至10岁的孩子设计，旨在为他们提供一个有趣的艺术启蒙体
验。

块 2 内容:
002
问题：这门课都教些什么呢？
回答：课程内容包括绘画、手工制作、色彩理论和艺术欣赏等，帮助孩子们全方位了解艺术。

块 3 内容:
003
问题：授课是怎么进行的？是大班还是小班？
回答：我们采用小班授课，注重互动与实践，让每个孩子都能得到充分的关注与指导。

块 4 内容:
004
问题：孩子上课需要带自己的材料吗？
回答：大部分材料由我们提供，但建议孩子自带一些基础工具，如水彩笔和素描本，以增强
个人创作体验。

块 5 内容:
005
问题：上课时间是什么时候？多久上一次课？
回答：每节课通常为1.5小时，每周上一次课，具体时间可根据家长和孩子的需求进行调整。

块 6 内容:
006
问题：老师很专业吗？有什么资格？
回答：我们的教师均具有专业的美术教育背景，并拥有丰富的儿童艺术教学经验。

块 7 内容:
007
问题：课程费用贵不贵？一般多少？
回答：课程费用根据不同班级和课程时长而定，请联系我们获取详细的价格信息。

块 8 内容:
008
问题：可以先试听一节课吗？要怎么预约？
回答：我们欢迎家长和孩子来参加免费的试听课，您可以提前预约。

块 9 内容:
009
问题：孩子完成课程后，可以展示自己的作品吗？
回答：是的，我们会定期举办孩子们的作品展览，让他们有机会展示自己的创作成果。

块 10 内容:
010
问题：如果孩子缺课，有补课的机会吗？
回答：当然可以！我们将为缺课的孩子提供补课选项，以确保他们能够跟上进度。



In [7]:
# 打印最后一个块
print(f"最后一个块的内容:\n{qa_chunks[-1]}")

最后一个块的内容:
250
问题：你们的课程适合未来想当艺术家的孩子吗？
回答：非常适合！我们的课程为孩子们打下良好的艺术基础，也为未来的艺术之路奠定信心。


### **Step3 存储嵌入**

#### **使用FAISS**
**FAISS**：与chroma类似，都是嵌入式向量数据库，由facebook提供
<br>**API文档**: [Langchain FAISS API](https://python.langchain.com/v0.2/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html)

In [8]:
# 使用faiss向量存储和OpenAIEmmbeddings模型将分割的文本块嵌入并存储
from langchain.vectorstores import FAISS

# 使用 FAISS.from_texts()存储
vectorstore_faiss = FAISS.from_texts(
    texts=qa_chunks,  # 使用分割后的文档块
    embedding=OpenAIEmbeddings()
)

In [9]:
# 查看 vectorstore_faiss 数据类型
type(vectorstore_faiss) 

langchain_community.vectorstores.faiss.FAISS

### **Step 4 检索文档**

#### **检索FAISS库**

In [10]:
# 使用 VectorStoreRetriever 从向量存储中检索与查询最相关的文档
retriever_faiss = vectorstore_faiss.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [11]:
type(retriever_faiss)

langchain_core.vectorstores.base.VectorStoreRetriever

In [12]:
retriever_faiss_docs = retriever_faiss.invoke("包含哪些课程内容？")

In [13]:
# 检查检索到的文本内容
print(len(retriever_faiss_docs))

6


In [14]:
print(retriever_faiss_docs[0].page_content)

002
问题：这门课都教些什么呢？
回答：课程内容包括绘画、手工制作、色彩理论和艺术欣赏等，帮助孩子们全方位了解艺术。


### **Step5 生成回答**

In [15]:
# 定义 RAG 链，将用户问题与检索到的文档结合并生成答案
llm = ChatOpenAI(model="gpt-4o-mini")

In [16]:
# 使用 hub 模块拉取 rag 提示词模板
prompt = hub.pull("rlm/rag-prompt")

D:\anacondaInstall\envs\langchain\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [17]:
# 打印模板
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [18]:
# 为 context 和 question 填充样例数据，并生成 ChatModel 可用的 Messages
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [19]:
# 查看提示词
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


### **提出3个问题，测试本地文档召回情况**

In [20]:
# 定义格式化文档的函数
def format_docs_faiss(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [21]:
# 使用LCEL 构建 RAG CHAIN
rag_chain_faiss = (
    {"context":retriever_faiss | format_docs_faiss, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
# 流式生成回答
for chunk in rag_chain_faiss.stream("课程怎么收费？"):
    print(chunk,end="",flush=True)

课程收费根据不同班级和课程时长而定，具体价格信息请与我们联系。我们也会不定期推出优惠活动，欢迎关注我们的官网或社交媒体获取最新信息。

In [23]:
for chunk in rag_chain_faiss.stream("课程都包含哪些内容？"):
    print(chunk,end="",flush=True)

课程内容包括绘画、手工制作、色彩理论和艺术欣赏，帮助孩子们全方位了解艺术。此外，课程还会融入艺术史知识和跨学科内容，如科学和历史，以增强理解。学习流程包括热身练习、主题创作和作品评析分享。

In [24]:
for chunk in rag_chain_faiss.stream("孩子学不会怎么办"):
    print(chunk,end="",flush=True)

如果孩子在学习中感到挫折，可以通过积极的反馈和鼓励来帮助他们，强调每次尝试都是成长的机会，而不必过于关注结果。同时，个别指导也可以提供，确保每位孩子都能得到适合自己的教学。

### **自定义提示词模版**

In [25]:
from langchain_core.prompts import PromptTemplate

# 自定义提示词模版
custom_template ="""
请仅根据以下内容回答最后的问题。
不要使用任何未提到的额外信息，如果找不到答案，请直接说不知道，不要尝试推测或编造答案。
答案最多使用三句话，并保持简洁明了。
请在回答结束时加上“谢谢提问！”。

{context}

问题：{question}

基于以上信息的答案：
"""
custom_rag_prompt = PromptTemplate.from_template(custom_template)

In [26]:
# # 为 context 和 question 填充样例数据，生成 LLM 可用的提示词
print(custom_rag_prompt.invoke({"context": "filler context", "question": "filler question"}).text)


请仅根据以下内容回答最后的问题。
不要使用任何未提到的额外信息，如果找不到答案，请直接说不知道，不要尝试推测或编造答案。
答案最多使用三句话，并保持简洁明了。
请在回答结束时加上“谢谢提问！”。

filler context

问题：filler question

基于以上信息的答案：



In [27]:
# 重新自定义 RAG Chain
custom_rag_chain = (
    {"context": retriever_faiss | format_docs_faiss, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [28]:
# 使用自定义prompt生成回答
for chunk in custom_rag_chain.stream("每周上几次课"):
    print(chunk,end="",flush=True)

每周上一次课。谢谢提问！

In [29]:
for chunk in custom_rag_chain.stream("中途可以退费么"):
    print(chunk,end="",flush=True)

不知道。谢谢提问！

In [30]:
for chunk in custom_rag_chain.stream("是否会讲述大师级的作品"):
    print(chunk,end="",flush=True)

是的，课程中会讲述一些著名艺术家的作品，如毕加索和达利等。谢谢提问！